In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

pd.plotting.register_matplotlib_converters()

# Paso 1: Cargar y preprocesar el conjunto de datos
url = "https://raw.githubusercontent.com/lespinozahuiman/Datasets/main/DataSet_TF_v5.csv"
df = pd.read_csv(url)

# Convertir la columna 'InvoiceDate' a formato datetime y extraer el mes y el año
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['periodo'] = df['InvoiceDate'].dt.to_period('M')

df['periodo'].head()

# Filtrar el conjunto de datos para incluir solo los periodos de interés (2021-10, 2021-11, 2021-12)
periodos_historicos = ['2021-10', '2021-11', '2021-12']
df_historico = df[df['periodo'].astype(str).isin(periodos_historicos)]

df_historico['periodoId'] = df_historico.groupby('periodo').ngroup()

df_historico



/var/folders/m2/030rdksn1yg5r7l01hsf80bh0000gn/T/ipykernel_19821/1266638965.py:12: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
/var/folders/m2/030rdksn1yg5r7l01hsf80bh0000gn/T/ipykernel_19821/1266638965.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_historico['periodoId'] = df_historico.groupby('periodo').ngroup()


,Retailer,RetailerID,InvoiceDate,Region,State,City,Product,PricePerUnit,UnitsSold,TotalSales,OperatingProfit,SalesMethod,periodo,periodoId
5,West Gear,1185732,2021-11-01,Midwest,Missouri,St. Louis,Women's Street Footwear,15.0,475.0,7125.0,2494.0,In-store,2021-11,1
6,Sports Direct,1185732,2021-11-17,South,Mississippi,Jackson,Men's Athletic Footwear,20.0,450.0,9000.0,3150.0,Online,2021-11,1
11,West Gear,1185732,2021-11-02,Midwest,Missouri,St. Louis,Women's Street Footwear,15.0,425.0,6375.0,2231.0,In-store,2021-11,1
12,West Gear,1185732,2021-11-06,West,Utah,Salt Lake City,Women's Street Footwear,20.0,425.0,8500.0,2975.0,In-store,2021-11,1
13,Amazon,1185732,2021-10-04,Midwest,Ohio,Columbus,Women's Street Footwear,20.0,425.0,8500.0,3400.0,Online,2021-10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9635,Foot Locker,1185732,2021-12-25,Northeast,Pennsylvania,Philadelphia,Men's Apparel,70.0,150.0,10500.0,3675.0,Outlet,2021-12,2
9636,Amazon,1185732,2021-11-18,Northeast,Maine,Portland,Men's Apparel,65.0,150.0,9750.0,2438.0,Outlet,2021-11,1
9637,Amazon,1185732,2021-10-22,West,Alaska,Anchorage,Women's Athletic Footwear,65.0,150.0,9750.0,2438.0,Outlet,2021-10,0
9638,Amazon,1185732,2021-10-22,West,Alaska,Anchorage,Men's Apparel,75.0,150.0,11250.0,2813.0,Outlet,2021-10,0


In [23]:
# Crear la variable objetivo y las características
X = df_historico[['periodoId']]
y = df_historico['TotalSales']

# Paso 3: Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Entrenar un modelo XGBoost
model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100)
model.fit(X_train, y_train)

# Paso 5: Utilizar el modelo entrenado para predecir el TotalSale para el siguiente periodo (2022-01)
siguiente_periodo = pd.Period('2022-02')
siguiente_periodo_numerico = siguiente_periodo.year * 100 + siguiente_periodo.month
prediccion_siguiente_periodo = model.predict([[siguiente_periodo_numerico]])

print("Predicción TotalSale para el siguiente periodo (2022-01):", prediccion_siguiente_periodo[0])

Predicción TotalSale para el siguiente periodo (2022-01): 9134.423
